In [172]:
import pandas as pd
import numpy as np
import xgboost as xgb
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score
import sklearn




In [173]:
train = pd.read_csv('/Users/mk/Downloads/titanic/train.csv')
test = pd.read_csv('/Users/mk/Downloads/titanic/test.csv')



In [174]:
# def impute_age(df):
#     #Imputes age using MissForest algorithm
#     features = ['Pclass','Age','SibSp','Parch', 'Sex']
#     df_impute = pd.get_dummies(df[features])
#     imputer = MissForest()
#     output_array = imputer.fit_transform(df_impute)
#     output_dataframe = pd.DataFrame(output_array, columns = df_impute.columns.values.tolist())
#     df['Age'] = output_dataframe['Age']
#     df['Age'] = df['Age'].fillna(df['Age'].mean())

def impute_age(df):   
    #Imputes missing age values using the mean
    df['Age'] = df['Age'].fillna(df['Age'].mean())
    df['Age'] = df['Age'].astype(int) # trims decimals

def impute_fare_embarked(df):
    #Imputes missing values for Fare and Port of Embarkation
    df['Fare'] = df['Fare'].fillna(0)
    df['Fare'] = df['Fare'].astype(int)
    df['Embarked'] = df['Embarked'].fillna('S')

def calc_fare_per_passenger(df):
    #Adjusts fare to be per person as opposed to per family 
    df['Relatives'] = df['SibSp'] + df['Parch'] 
    df['Fare_Per_Passenger'] = df['Fare']/(df['Relatives']+1)
    df['Fare_Per_Passenger'] = df['Fare_Per_Passenger'].astype(int)
    
def drop_features(df):
    #Drops unused features and features with high correlation
    df.drop(['Cabin','Ticket', 'Name', 'SibSp', 'Parch', 'Fare'], axis = 1, inplace = True)
    
def drop_features2(df):
    #Drops unused features and features with high correlation
    df.drop(['Cabin','Ticket', 'Name'], axis = 1, inplace = True)

def calc_family_size(df):
    #Calculates size of family including self
    df['FamilySize'] = df['SibSp'] +  df['Parch'] + 1
    
def calc_alone(df): 
    #Checks if traveler was alone
    df['IsAlone'] = 0
    df.loc[df['FamilySize'] == 1, 'IsAlone'] = 1  

    

In [175]:
def final(df):
    impute_age(df)
    impute_fare_embarked(df)
#     calc_fare_per_passenger(df)
#     calc_family_size(df)
#     calc_alone(df)
#     drop_features(df)
    drop_features2(df)
    

In [176]:
final(train)
final(test)

In [177]:
y_train = train['Survived']
train_drop = train.drop(['Survived','PassengerId'], axis = 1)
columns1 = list(train_drop)

X_train = pd.get_dummies(train_drop[columns1])

test_drop = test.drop(['PassengerId'], axis = 1)
columns2 = list(train_drop)

X_test = pd.get_dummies(test_drop[columns2])





In [178]:
# XGBoost Tuning
def XGB_cl_bo(max_depth, learning_rate, min_child_weight, n_estimators, subsample):
    params_gbm = {}
    params_gbm['max_depth'] = round(max_depth)
    params_gbm['min_child_weight'] = min_child_weight
    params_gbm['learning_rate'] = learning_rate
    params_gbm['n_estimators'] = round(n_estimators)
    params_gbm['subsample'] = subsample
    scores = cross_val_score(XGBClassifier(**params_gbm),
                             X_train, y_train, scoring='accuracy', cv=5)
    score = scores.mean()
    return score
# Run Bayesian Optimization
params_gbm ={
    'max_depth':(1, 5),
    'min_child_weight':(1, 6),
    'learning_rate':(0.01, 1),
    'n_estimators':(10, 500),
    'subsample': (0.4, 1)
}
gbm_bo = BayesianOptimization(XGB_cl_bo, params_gbm)
gbm_bo.maximize(init_points=30, n_iter=300)

params_gbm = gbm_bo.max['params']
params_gbm['max_depth'] = round(params_gbm['max_depth'])
params_gbm['n_estimators'] = round(params_gbm['n_estimators'])
params_gbm



|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... | subsample |
-------------------------------------------------------------------------------------
| 1         | 0.8149    | 0.2469    | 2.151     | 5.169     | 33.63     | 0.8249    |
| 2         | 0.8182    | 0.1862    | 3.868     | 2.477     | 211.0     | 0.7472    |
| 3         | 0.8194    | 0.425     | 3.129     | 3.835     | 396.4     | 0.6427    |
| 4         | 0.8205    | 0.7147    | 1.908     | 4.278     | 463.2     | 0.7642    |
| 5         | 0.8294    | 0.1519    | 2.412     | 4.323     | 475.0     | 0.5223    |
| 6         | 0.816     | 0.6103    | 2.604     | 4.456     | 376.1     | 0.7705    |
| 7         | 0.8216    | 0.107     | 1.917     | 5.056     | 303.1     | 0.8857    |
| 8         | 0.8092    | 0.7148    | 3.236     | 1.575     | 185.7     | 0.4894    |
| 9         | 0.7969    | 0.7397    | 4.643     | 1.395     | 426.5     | 0.4623    |
| 10        | 0.8205    | 0.3847    | 2.204     | 2.42

| 58        | 0.8283    | 0.1913    | 3.132     | 5.562     | 123.9     | 0.9615    |
| 59        | 0.8362    | 0.2923    | 2.162     | 1.968     | 145.1     | 0.8348    |
| 60        | 0.8249    | 0.8264    | 2.02      | 3.005     | 188.4     | 0.9761    |
| 61        | 0.8373    | 0.3889    | 1.799     | 3.962     | 268.9     | 0.8238    |
| 62        | 0.8283    | 0.2152    | 1.767     | 4.335     | 268.8     | 0.9236    |
| 63        | 0.8216    | 0.5105    | 4.692     | 2.465     | 250.0     | 0.4183    |
| 64        | 0.8081    | 0.5758    | 3.808     | 2.68      | 176.2     | 0.644     |
| 65        | 0.8294    | 0.0509    | 3.629     | 4.076     | 275.2     | 0.5857    |
| 66        | 0.8261    | 0.2228    | 4.802     | 2.204     | 166.0     | 0.6554    |
| 67        | 0.8193    | 0.2305    | 2.664     | 2.29      | 66.47     | 0.8035    |
| 68        | 0.8406    | 0.06531   | 4.512     | 5.011     | 440.7     | 0.8012    |
| 69        | 0.8328    | 0.2423    | 3.573     | 5.61

| 116       | 0.8361    | 0.2752    | 4.69      | 3.06      | 77.71     | 0.5443    |
| 117       | 0.8328    | 0.3658    | 3.177     | 5.895     | 396.9     | 0.9483    |
| 118       | 0.8339    | 0.2219    | 4.056     | 5.326     | 396.5     | 0.6739    |
| 119       | 0.8003    | 0.7702    | 1.115     | 3.833     | 121.6     | 0.8451    |
| 120       | 0.825     | 0.192     | 3.297     | 5.323     | 123.6     | 0.9617    |
| 121       | 0.8238    | 0.4045    | 4.441     | 5.567     | 358.5     | 0.7905    |
| 122       | 0.798     | 0.4511    | 4.368     | 4.035     | 494.0     | 0.8974    |
| 123       | 0.8294    | 0.3148    | 3.729     | 5.266     | 328.1     | 0.9371    |
| 124       | 0.8205    | 0.2158    | 1.986     | 1.431     | 451.9     | 0.6213    |
| 125       | 0.8306    | 0.2259    | 2.617     | 5.754     | 123.5     | 0.764     |
| 126       | 0.8171    | 0.4408    | 2.981     | 1.969     | 337.7     | 0.6248    |
| 127       | 0.8317    | 0.1981    | 3.361     | 5.33

| 174       | 0.8126    | 0.1896    | 4.689     | 1.238     | 409.7     | 0.6637    |
| 175       | 0.8384    | 0.1585    | 2.296     | 2.904     | 376.6     | 0.6339    |
| 176       | 0.8261    | 0.178     | 1.914     | 2.947     | 376.4     | 0.7374    |
| 177       | 0.8328    | 0.2195    | 2.269     | 3.071     | 376.5     | 0.6093    |
| 178       | 0.8294    | 0.4798    | 2.725     | 4.841     | 109.8     | 0.8561    |
| 179       | 0.8227    | 0.96      | 2.735     | 4.821     | 137.2     | 0.6297    |
| 180       | 0.8384    | 0.117     | 3.573     | 5.643     | 397.0     | 0.8853    |
| 181       | 0.798     | 0.1397    | 1.424     | 5.205     | 154.0     | 0.507     |
| 182       | 0.8328    | 0.07626   | 4.406     | 5.337     | 440.4     | 0.6914    |
| 183       | 0.8317    | 0.3868    | 2.512     | 4.276     | 109.8     | 0.8044    |
| 184       | 0.8328    | 0.06561   | 3.75      | 5.269     | 328.1     | 0.645     |
| 185       | 0.8272    | 0.9932    | 1.952     | 3.14

| 232       | 0.8194    | 0.8751    | 2.702     | 2.631     | 56.74     | 0.8788    |
| 233       | 0.8205    | 0.2682    | 3.514     | 3.46      | 442.5     | 0.4414    |
| 234       | 0.8014    | 0.5736    | 3.048     | 2.834     | 282.7     | 0.5486    |
| 235       | 0.8361    | 0.5867    | 2.875     | 5.369     | 123.9     | 0.9861    |
| 236       | 0.8104    | 0.5126    | 3.08      | 2.887     | 321.9     | 0.5593    |
| 237       | 0.807     | 0.8774    | 1.302     | 4.154     | 80.07     | 0.9827    |
| 238       | 0.8305    | 0.6013    | 2.825     | 4.272     | 109.5     | 0.5615    |
| 239       | 0.8317    | 0.3109    | 2.806     | 2.645     | 376.5     | 0.5847    |
| 240       | 0.8317    | 0.04865   | 4.16      | 5.744     | 397.3     | 0.8831    |
| 241       | 0.8361    | 0.208     | 3.486     | 5.994     | 342.0     | 0.7215    |
| 242       | 0.8328    | 0.1016    | 3.558     | 5.764     | 396.6     | 0.7455    |
| 243       | 0.7958    | 0.8551    | 4.492     | 1.41

| 290       | 0.8137    | 0.01013   | 2.313     | 3.024     | 376.3     | 0.767     |
| 291       | 0.8261    | 0.4583    | 3.395     | 5.891     | 397.3     | 0.7206    |
| 292       | 0.8204    | 0.4472    | 2.796     | 4.444     | 314.8     | 0.5201    |
| 293       | 0.8294    | 0.287     | 4.461     | 2.849     | 77.49     | 0.7387    |
| 294       | 0.8149    | 0.7564    | 3.292     | 5.703     | 383.0     | 0.6811    |
| 295       | 0.8283    | 0.09863   | 4.779     | 4.842     | 123.2     | 0.8367    |
| 296       | 0.8216    | 0.5458    | 4.889     | 3.476     | 266.8     | 0.6156    |
| 297       | 0.8092    | 0.3099    | 4.991     | 2.367     | 433.2     | 0.7562    |
| 298       | 0.8182    | 0.324     | 4.369     | 4.297     | 328.3     | 0.9502    |
| 299       | 0.8306    | 0.3677    | 4.131     | 5.916     | 396.5     | 0.5666    |
| 300       | 0.8216    | 0.1238    | 3.826     | 1.16      | 203.1     | 0.7505    |
| 301       | 0.8373    | 0.08712   | 3.121     | 4.11

{'learning_rate': 0.062023124880840115,
 'max_depth': 5,
 'min_child_weight': 4.9153226099037415,
 'n_estimators': 441,
 'subsample': 0.8657174823569895}

In [179]:
# output = pd.DataFrame({'Survived': predictions}, index = test.PassengerId.astype(int))
# output.to_csv('Titanic_0404.csv')

In [180]:
# {'learning_rate': 0.01,
#  'max_depth': 9,
#  'min_child_weight': 1.7527629096237887,
#  'n_estimators': 202,
#  'subsample': 0.7827776208670194}